In [ ]:
!pip install wandb seqeval --quiet
!pip install protobuf==3.20.*

In [1]:
from kaggle_secrets import UserSecretsClient
import wandb

# Load your secret
user_secrets = UserSecretsClient()
wandb_key = user_secrets.get_secret("WANDB_API_KEY")

# Login
wandb.login(key=wandb_key)

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

True

In [2]:
import numpy as np
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForTokenClassification,
    Trainer, 
    TrainingArguments
)
from datasets import load_dataset
from seqeval.metrics import classification_report, f1_score
import wandb

# Initialize W&B
wandb.init(
    project="biomedical-ner",
    name="pubmedbert-maccrobat-v1",
    tags=["pubmedbert", "maccrobat", "baseline"]
)

print("✅ W&B initialized")

2025-11-13 17:46:02.152213: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763055962.390832     107 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763055962.451725     107 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


✅ W&B initialized


In [3]:
# Load MACCROBAT from HuggingFace
dataset = load_dataset("ktgiahieu/maccrobat2018_2020")

# Check structure
print(f"Train examples: {len(dataset['train'])}")
print(f"Example structure:")
example = dataset['train'][0]
print(f"  Keys: {example.keys()}")
print(f"  Tokens: {example['tokens'][:10]}")
print(f"  Tags: {example['tags'][:10]}")

# Get unique tags for num_labels
unique_tags = set()
for example in dataset['train']:
    unique_tags.update(example['tags'])

# Ensure every B-XXX tag has a corresponding I-XXX tag
extra_i_tags = set()
for tag in unique_tags:
    if tag.startswith("B-"):
        i_tag = "I-" + tag[2:]
        if i_tag not in unique_tags:
            extra_i_tags.add(i_tag)
unique_tags.update(extra_i_tags)

unique_tags = sorted(list(unique_tags))
num_labels = len(unique_tags)
tag2id = {tag: i for i, tag in enumerate(unique_tags)}
id2tag = {i: tag for tag, i in tag2id.items()}

print(f"\n✅ Found {num_labels} unique entity types")
print(f"Entity types: {unique_tags[:10]}...")  # Show first 10

README.md: 0.00B [00:00, ?B/s]

data.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/400 [00:00<?, ? examples/s]

Train examples: 400
Example structure:
  Keys: dict_keys(['tokens', 'tags'])
  Tokens: ['A', '68', '-', 'year', '-', 'old', 'female', 'nonsmoker', ',', 'nondrinker']
  Tags: ['O', 'B-Age', 'I-Age', 'I-Age', 'I-Age', 'I-Age', 'B-Sex', 'B-History', 'O', 'B-History']

✅ Found 83 unique entity types
Entity types: ['B-Activity', 'B-Administration', 'B-Age', 'B-Area', 'B-Biological_attribute', 'B-Biological_structure', 'B-Clinical_event', 'B-Color', 'B-Coreference', 'B-Date']...


In [4]:
# Load tokenizer
model_name = "microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_and_align_labels(examples):
    """
    Tokenize text and align labels with tokens.
    
    Key insight: BERT tokenizes words into subwords (e.g., "myocardial" → ["my", "##ocardial"]).
    We need to align BIO tags with these subword tokens.
    
    Strategy: 
    - First subword of a word gets the original tag (B-DISEASE)
    - Continuation subwords get I-tag version (I-DISEASE)
    - Special tokens ([CLS], [SEP]) get ignored (label_id = -100)
    """
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,  # Already pre-tokenized
        max_length=512,
        padding=False  # We'll handle padding in data collator
    )
    
    labels = []
    for i, label in enumerate(examples["tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_idx = None
        
        for word_idx in word_ids:
            if word_idx is None:
                # Special tokens → ignore in loss
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                # First subword of a word → use original label
                label_ids.append(tag2id[label[word_idx]])
            else:
                # Continuation subwords → convert B-tag to I-tag
                original_tag = label[word_idx]
                if original_tag.startswith("B-"):
                    # B-DISEASE → I-DISEASE
                    label_ids.append(tag2id["I-" + original_tag[2:]])
                else:
                    # Keep I-tags as-is
                    label_ids.append(tag2id[original_tag])
            
            previous_word_idx = word_idx
        
        labels.append(label_ids)
    
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Apply tokenization to dataset
print("Tokenizing dataset...")
tokenized_dataset = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset["train"].column_names
)

print(f"✅ Tokenized train examples: {len(tokenized_dataset['train'])}")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Tokenizing dataset...


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

✅ Tokenized train examples: 400


In [5]:
from transformers import DataCollatorForTokenClassification

# Data collator pads sequences to same length in each batch
data_collator = DataCollatorForTokenClassification(tokenizer)

print("✅ Data collator created")

✅ Data collator created


In [21]:
def compute_metrics(p):
    """
    Compute standard NER metrics: precision, recall, F1.
    Uses seqeval for entity-level evaluation.
    """
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    
    # Remove -100 labels (special tokens we ignored)
    true_predictions = [
        [id2tag[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2tag[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    # Calculate metrics
    results = {
        "precision": f1_score(true_labels, true_predictions, average="macro"),
        "recall": f1_score(true_labels, true_predictions, average="macro"),
        "f1": f1_score(true_labels, true_predictions, average="micro")
    }
    
    return results

print("✅ Metrics function created")

✅ Metrics function created


In [7]:
# Load PubMedBERT with token classification head
model_name = "microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext"
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=num_labels
)

print(f"✅ Model loaded: {model_name}")
print(f"   Parameters: {model.num_parameters():,}")
print(f"   Entity types: {num_labels}")

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Model loaded: microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext
   Parameters: 108,955,475
   Entity types: 83


In [22]:
training_args = TrainingArguments(
    output_dir="./pubmedbert-maccrobat",

    # Data
    per_device_train_batch_size=16,  # Kaggle T4 can handle this
    per_device_eval_batch_size=32,

    # Epochs & steps
    num_train_epochs=3,  # Start with 3, increase if needed
    eval_strategy="epoch",  # Evaluate after each epoch

    # Learning rate
    learning_rate=2e-5,  # Standard for fine-tuning BERT
    warmup_steps=500,   # Linear warmup for first 500 steps
    weight_decay=0.01,

    # Optimization
    optim="adamw_torch",
    gradient_accumulation_steps=1,
    max_grad_norm=1.0,

    # Logging & saving
    logging_steps=100,  # Log metrics every 100 steps
    save_strategy="epoch",  # Save model after each epoch
    load_best_model_at_end=True,
    metric_for_best_model="f1",

    # W&B integration
    report_to=["wandb"],
    run_name="pubmedbert-maccrobat-v1",

    # Misc
    seed=42,
    fp16=True,  # Use mixed precision to save memory on T4
)

print("✅ Training arguments configured")

✅ Training arguments configured


In [16]:
from datasets import DatasetDict

# Split train set into train/validation (e.g., 90% train, 10% val)
split_dataset = tokenized_dataset["train"].train_test_split(test_size=0.1, seed=42)
tokenized_dataset = DatasetDict({
    "train": split_dataset["train"],
    "validation": split_dataset["test"]
})

print(f"Train size: {len(tokenized_dataset['train'])}")
print(f"Validation size: {len(tokenized_dataset['validation'])}")

Train size: 360
Validation size: 40


In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

print("✅ Trainer initialized")

✅ Trainer initialized


/tmp/ipykernel_107/3751207074.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [24]:
# Train!
print("Starting training...")
print(f"Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

train_result = trainer.train()

print("\n✅ Training complete!")
print(f"Final loss: {train_result.training_loss:.4f}")

Starting training...
Device: Tesla T4


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,4.491906,0.001358,0.001358,0.004616
2,No log,4.427600,0.001392,0.001392,0.004710
3,No log,4.316536,0.001468,0.001468,0.004898


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



✅ Training complete!
Final loss: 4.4471


In [25]:
# Evaluate on training set (TODO: use actual validation set)
print("Evaluating...")
eval_results = trainer.evaluate()

print("\nEvaluation Results:")
for key, value in eval_results.items():
    print(f"  {key}: {value:.4f}")

# Save final model
trainer.save_model("./pubmedbert-maccrobat-final")
print("\n✅ Model saved to ./pubmedbert-maccrobat-final")

# Log final metrics to W&B
wandb.log(eval_results)
wandb.finish()

Evaluating...


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



Evaluation Results:
  eval_loss: 4.3165
  eval_precision: 0.0015
  eval_recall: 0.0015
  eval_f1: 0.0049
  eval_runtime: 0.8635
  eval_samples_per_second: 46.3240
  eval_steps_per_second: 1.1580
  epoch: 3.0000

✅ Model saved to ./pubmedbert-maccrobat-final


epoch,▁
eval/f1,▁▃██
eval/loss,█▅▁▁
eval/precision,▁▃██
eval/recall,▁▃██
eval/runtime,▁▂▅█
eval/samples_per_second,█▇▄▁
eval/steps_per_second,█▇▄▁
eval_f1,▁
eval_loss,▁
eval_precision,▁


In [29]:
# Load the trained model
from transformers import pipeline

# Reload best model
best_model = AutoModelForTokenClassification.from_pretrained("./pubmedbert-maccrobat-final")
nlp = pipeline(
    "token-classification",
    model=best_model,
    tokenizer=tokenizer,
    aggregation_strategy="simple"
)

# Test on a sample
test_text = "A 67-year-old female presented with acute myocardial infarction and received aspirin."
results = nlp(test_text)

print("Test Inference:")
print(f"Input: {test_text}\n")
for result in results:
    print(f"  {result['word']:30s} → {result['entity_group']:30s} (conf: {result['score']:.3f})")

Device set to use cuda:0


Test Inference:
Input: A 67-year-old female presented with acute myocardial infarction and received aspirin.

  a                              → LABEL_60                       (conf: 0.020)
  67                             → LABEL_66                       (conf: 0.022)
  - year - old                   → LABEL_60                       (conf: 0.023)
  female                         → LABEL_56                       (conf: 0.020)
  presented                      → LABEL_60                       (conf: 0.020)
  with                           → LABEL_56                       (conf: 0.021)
  acute                          → LABEL_36                       (conf: 0.021)
  myocardial infarction          → LABEL_56                       (conf: 0.022)
  and                            → LABEL_52                       (conf: 0.019)
  received                       → LABEL_60                       (conf: 0.020)
  aspirin                        → LABEL_52                       (conf: 0.020)
  .       